# Stability of the AMOC

In [ ]:
import os
import sys
import cmocean
import numpy as np
import xarray as xr
import warnings
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("..")
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

## $M_{ov}$ calculation

The fundamental idea behind the $M_{ov}$ parameter is the effect of a MOC shutdown has on the salinity in the North Atlantic and hence NADW formation.
If the overturning circulation imports freshwater to the Atlantic, a MOC shutdown will increase salinity which counteracts the shutdown.
Vice versa, a freshwater export due to the MOC will lead to freshening of the North Atlantic in case of a shutdown and hence a strengthening of the shut down state.

The Atlantic freshwater budget in steady state is

$$\left[E - P - R \right] = M_{ov} + M_{az} + M_{dif} + M_{BS}$$

with evaporation $E$, precipitation $P$, runoff $R$, as well as freshwater transport terms due to the overturning circulation $M_{ov}$, the azonal gyre circulation $M_{az}$, diffusion $M_{dif}$, and exchange via the Bering Strait $M_{BS}$.


The overturning freswater transport term $M_{ov}$ is given differently in different publications:

_de Vries et al. (2005)_:

$$M_{ov} = - \frac{1}{S_0} \int \bar{v}(z) \left[ \langle S(z) \rangle - S_0 \right] \,\mathrm{d}z$$

where $\bar{v}(z) = \int v(x,z) \,\mathrm{d}x$ is the zonal integral and $\langle S \rangle (z) = \int S(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x$ is the zonal mean, such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} - S_0 \right] \,\mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} \right] \,\mathrm{d}z + \int \int v(x,z) \, \mathrm{d}x \mathrm{d}z  \tag{1}
\end{align}

_Mecking et al. (2017)_:

$$M_{ov} = - \frac{1}{S_0} \int \int v^*(z) \langle S(z) \rangle \, \mathrm{d}x \mathrm{d}z$$

where $v^*(z) = \langle v \rangle (z) - \bar{v} = \int v(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x - \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z$ such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \int \left[ \frac{\int v(x,z) \,\mathrm{d}x}{ \int \mathrm{d}x} - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}x \mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int  \left[ \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int \left[ \frac{ \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \times   \int S(x,z) \,\mathrm{d}x }{\int \int \mathrm{d}x \mathrm{d}z } \right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \left[ \frac{1}{S_0} \frac{  \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z } \right] \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \tag{2}
\end{align}

Equations (2) and (1) are equal if the reference salinity is equal to the section average: $S_0 = \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z $

In [ ]:
from MOC import approx_lats
from tqdm import tqdm_notebook
from paths import file_ex_ocn_ctrl, file_ex_ocn_rcp, file_ex_ocn_lpd, path_prace, path_results, file_RMASK_ocn, file_RMASK_ocn_low
from filters import lowpass
from constants import rho_sw  # [kg/m^3]
from timeseries import IterateOutputCESM
from xr_regression import xr_lintrend, xr_linear_trend, xr_linear_trends_2D
from xr_DataArrays import xr_DZ

## Section at 34$^\circ$S

In [ ]:
# high resolution coordinates
j_34 = 810               # latitude number at the Cape of Good Hope
i_SA, i_CGH = 555, 1299  # longitude numbers of South America and the Cape of Good Hope

ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['VVEL', 'SALT', 'DXT', 'DXU', 'z_t', 'dz']].isel({'time':0, 'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
S0 = (ds.SALT*ds.DXT*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.SALT>0)*ds.dz).sum(dim=['nlon','z_t'])
v0 = (ds.VVEL*ds.DXU*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.VVEL>-1000)*ds.dz).sum(dim=['nlon','z_t'])
L = ds.DXT.sum(dim='nlon')
print(f'S_0 = {S0.values:6.2f} g/kg \nv_0 = {v0.values:6.3f} cm/s \nL   = {L.values/1e5:6.0f} km')

In [ ]:
ds.TLONG[ds.TLONG>180] -= 360

X, Y = np.meshgrid(ds.TLONG, -ds.z_t/1e5)
v0_str, S0_str = r'$v_0$', r'$S_0$'

f,ax = plt.subplots(2,3, figsize=(8,7), sharey=True)
for i in range(2):
    ax[i,0].set_ylabel('depth [km]')
    ax[i,0].set_ylim((-5.5,0))
    for j in range(3):
        if j in [0, 2]:
            ax[i,j].set_xlim(-55,20)
            ax[i,j].set_xticks(np.arange(-40, 30, 20))
            ax[i,j].set_xticklabels(np.arange(-40, 30, 20))

ax[0,0].set_title(r'$S(y,z)$')
im0 = ax[0,0].pcolormesh(X, Y, ds.SALT, vmin=34, vmax=35.5, cmap='cmo.haline_r')
plt.colorbar(im0, ax=ax[0,0], orientation='horizontal')

ax[0,1].plot(ds.SALT.mean('nlon', skipna=True), -ds.z_t/1e5)
ax[0,1].axvline(S0, c='C1', label=f'{S0_str} = {S0.values:6.3f} g/kg')
ax[0,1].legend(frameon=False, handlelength=.8)
ax[0,1].set_title(r'$\langle S \rangle (z)$ and $S_0$')
ax[0,1].set_xlabel('salinity [g/kg]')

ax[0,2].set_title(r'$S(y,z) - \langle S \rangle (z)$')
im2 = ax[0,2].pcolormesh(X, Y, ds.SALT-ds.SALT.mean('nlon'), vmin=-.5, vmax=.5, cmap='RdBu_r')
plt.colorbar(im2, ax=ax[0,2], orientation='horizontal')


ax[1,0].set_title(r'$VVEL(y,z)$')
im0 = ax[1,0].pcolormesh(X, Y, ds.VVEL, vmin=-20, vmax=20, cmap='BrBG')
plt.colorbar(im0, ax=ax[1,0], orientation='horizontal')

ax[1,1].plot(ds.VVEL.mean('nlon', skipna=True), -ds.z_t/1e5)
ax[1,1].axvline(v0, c='C1', label=f'{v0_str} = {v0.values:6.4f} cm/s')
ax[1,1].legend(frameon=False, handlelength=.8, loc=3)
ax[1,1].set_title(r'$\langle VVEL \rangle (z)$ and $VVEL_0$')
ax[1,1].set_xlabel('salinity [g/kg]')

ax[1,2].set_title(r'$VVEL(y,z) - \langle VVEL \rangle (z)$')
im2 = ax[1,2].pcolormesh(X, Y, ds.VVEL-ds.VVEL.mean('nlon'), vmin=-10, vmax=10, cmap='RdBu_r')
plt.colorbar(im2, ax=ax[1,2], orientation='horizontal')

ax[0,0].text(-45,-0.5, 'thermocline waters')
ax[0,0].text(-45,-1.0, 'AAIW')
ax[0,0].text(-45,-2.5, 'NADW')
ax[0,0].text(-45,-4.5, 'AABW')

ax[1,1].arrow(x=.3, y=-.5, dx=.2, dy=0, width=.05, head_length=.1, head_width=.3, length_includes_head=True)
ax[1,1].arrow(x=.5, y=-2.5, dx=-.2, dy=0, width=.05, head_length=.1, head_width=.3, length_includes_head=True)
ax[1,1].set_xlim((-.3,.6))

plt.savefig(f'{path_results}/Mov/section34S_S_VVEL')

In [ ]:
def calc_Mov(ds, S0):
    # function given by de Vries et al. (2005)
    # can use .mean function because at 34S the grid is rectilinear
    Mov = ( -1/S0*(( ds.VVEL*ds.DXU*(ds.SALT.mean('nlon', skipna=True)-S0) )*ds.dz).sum(['nlon', 'z_t'])  )/ 1e12  # 1e12 cm^3/s = 1 Sv
    return Mov

def calc_Maz(ds, S0):
    # function given by de Vries et al. (2005)
    # can use .mean function because at 34S the grid is rectilinear
    Maz = ( -1/S0*(( ds.VVEL*ds.DXU*(ds.SALT.mean('nlon', skipna=True)-S0) )*ds.dz).sum(['nlon', 'z_t'])  )/ 1e12  # 1e12 cm^3/s = 1 Sv
    return Maz


# 56:16 for ctrl monthly
# 18:43 for rcp
warnings.simplefilter("ignore")
G = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['DXT', 'DXU', 'z_t', 'dz']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
for i, run in enumerate(['ctrl', 'rcp', 'lpd', 'lr1', 'lr2', 'ld']):
    fn_out = f'{path_prace}/Mov/Mov_monthly_{run}.nc'
    try:
        assert os.path.exists(fn_out)
        if run=='ctrl':  Mov_ctrl = xr.open_dataarray(fn_out, decode_times=False)
        if run=='rcp' :  Mov_rcp  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='lpd' :  Mov_lpd  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='lr1' :  Mov_lr1  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='lr2' :  Mov_lr2  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='ld'  :  Mov_ld   = xr.open_dataarray(fn_out, decode_times=False)
    except:
        if run in ['ctrl', 'rcp', 'hq']:
            roll, j_34, i_SA, i_CGH = 0, 810, 555, 1299
        if run in ['lpd', 'lr1', 'lr2', 'ld']:
            roll, j_34, i_SA, i_CGH = 100, 84, 88, 152
        for i, (y,m,fn) in tqdm_notebook(enumerate(IterateOutputCESM(run=run, domain='ocn', tavg='monthly'))):
            ds = xr.open_dataset(fn, decode_times=False)[['VVEL', 'SALT', 'DXT', 'DXU', 'z_t', 'dz']].roll(shifts={'nlon':roll}, roll_coords=True).isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
            S0 = (ds.SALT*ds.DXT*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.SALT>0)*ds.dz).sum(dim=['nlon','z_t'])
            Mov_ = calc_Mov(ds, S0)
            if i==0:  Mov = Mov_
            else:     Mov = xr.concat([Mov, Mov_], dim='time')
        Mov.to_netcdf(fn_out)

In [ ]:
f, ax = plt.subplots(1,2,figsize=(6.4,4), sharey=True, constrained_layout=True)
ax[0].set_ylabel(r'$M_{ov}$ [Sv]')
for i in range(2):  ax[i].axhline(c='grey', lw=.5)
# ax[0].set_xlim(-5,305)
for i, Mov in enumerate([Mov_ctrl, Mov_rcp]):
    ax[0].plot(Mov.time/365-[0,1800][i], Mov, lw=.5, alpha=.3)
    ax[0].plot(Mov.time[30:-30]/365-[0,1800][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['CTRL','RCP'][i])
    if i==1:
        ax[0].plot(Mov.time/365-1800, xr_lintrend(Mov)+.05, ls='--', c='C1')
        ax[0].text(200, -.2,f'{xr_linear_trend(Mov).values*365*100:.2f} Sv/100yr', color='C1', fontsize=8)


for i, Mov in enumerate([Mov_lpd, Mov_lr1, Mov_lr2, Mov_ld]):
    ax[1].plot(Mov.time/365-[0,1500,1380,1500][i], Mov, c=f'C{i}', lw=.5, alpha=.3)
    ax[1].plot(Mov.time[30:-30]/365-[0,1500,1380,1500][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['LPD', 'RCP1', 'RCP2', '2xC2'][i])
    if i in [1,2]:
        ax[1].plot(Mov.time/365-[0,1500,1380,1500][i], xr_lintrend(Mov)+.05, ls='--', c=f'C{i}')
        ax[1].text([500,620][i-1], .02,f'{xr_linear_trend(Mov).values*365*100:.2f}\nSv/100yr', color=f'C{i}', fontsize=8)
for i in range(2):
    ax[i].set_title(['HIGH CESM1.04', 'LOW CESM1.12'][i])
    ax[i].legend(frameon=False)
    ax[i].set_xlabel('time [model years]')
ax[1].errorbar(x=200, y=-.165, fmt='none', yerr=.115, capsize=3, c='k')
ax[1].text(220,-.165, 'range of observations\n[Weijer et al. (2019)]', va='center', fontsize=8)
plt.tight_layout()
plt.savefig(f'{path_results}/Mov/Mov_overview')

### RCP

In [ ]:
# # creating dataset of section
warnings.simplefilter("ignore")
# G = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['DXT', 'DXU', 'z_t', 'dz']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
for i, run in enumerate(['rcp']):  # 'ctrl'
    for j, (y,m,fn) in tqdm_notebook(enumerate(IterateOutputCESM(run=run, domain='ocn', tavg='monthly'))):
        if j==0:
            ds = xr.open_dataset(fn, decode_times=False)[['VVEL', 'SALT', 'DXT', 'DXU', 'z_t', 'dz']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
            TLAT_, TLONG_ = ds.TLAT, ds.TLONG
        else:
            ds_ = xr.open_dataset(fn, decode_times=False)[['VVEL', 'SALT']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
            ds_['TLAT'], ds_['TLONG'] = TLAT_, TLONG_
            ds = xr.merge([ds, ds_])#, compat='override')  # override results in empty fields
    ds.to_netcdf(f'{path_prace}/Mov/section_monthly_VVEL_SALT_{run}.nc')

In [ ]:
section_rcp = xr.open_dataset(f'{path_prace}/Mov/section_monthly_VVEL_SALT_rcp.nc', decode_times=False)

In [ ]:
# ca. 2:30 min
section_rcp_VVEL_mean = section_rcp.VVEL.isel(time=slice(0,12*30)).mean(dim='time')
section_rcp_SALT_mean = section_rcp.SALT.isel(time=slice(0,12*30)).mean(dim='time')
section_rcp_SALT_trend = xr_linear_trends_2D(da=section_rcp.SALT, dim_names=['z_t','nlon'],  with_nans=True)
section_rcp_VVEL_trend = xr_linear_trends_2D(da=section_rcp.VVEL, dim_names=['z_t','nlon'],  with_nans=True)

In [ ]:
section_rcp.TLONG[section_rcp.TLONG>180] -= 360
X, Y = np.meshgrid(section_rcp.TLONG, -section_rcp.z_t/1e5)
f, ax = plt.subplots(2, 3, figsize=(6.4,4.5), sharey=True, constrained_layout=True)

ax[0,2].plot(section_rcp_VVEL_mean.mean(dim='nlon'), -section_rcp.z_t/1e5, label=r'$\langle VVEL\rangle(z)$')
ax[0,2].plot(section_rcp_SALT_mean.mean(dim='nlon')-S0t.mean(dim='time'), -section_rcp.z_t/1e5, label=r'$\langle SALT\rangle(z)-S_0$')
ax[1,2].plot(section_rcp_VVEL_trend.mean(dim='nlon')*365*100, -section_rcp.z_t/1e5, label=r'$\frac{d}{dt}\langle VVEL\rangle$')
ax[1,2].plot(section_rcp_SALT_trend.mean(dim='nlon')*365*100, -section_rcp.z_t/1e5, label=r'$\frac{d}{dt}\langle SALT\rangle$')
for i in range(2):
    ax[i,2].legend(fontsize=8, frameon=False, handlelength=.3, loc=5)
    ax[i,2].set_xlabel(['[cm/s]\n[psu]', '[cm/s/100yr]\n[psu/100yr]'][i])
    ax[i,2].axvline(0, c='k', lw=.5)
    ax[0,i].set_title(['SALT', 'VVEL'][i])
    ax[i,0].set_ylabel(['mean', 'trend'][i]+'\ndepth [km]')
    for j in range(2):
        q = [[section_rcp_SALT_mean, section_rcp_VVEL_mean],[section_rcp_SALT_trend*365*100, section_rcp_VVEL_trend*365*100]][i][j]
        (minv, maxv) = [[(34,35.5),(-10,10)], [(-.6,.6), (-6,6)]][i][j]
        cmap = [['cmo.haline_r', 'RdBu_r'],['cmo.balance', 'cmo.balance']][i][j]
        label = [['[psu]', '[cm/s]'],['[psu/100yr]', '[cm/s/100yr]']][i][j]
        ax[i,j].set_ylim((-5.5,0))
        im0 = ax[i,j].pcolormesh(X, Y, q, vmin=minv, vmax=maxv, cmap=cmap)
        plt.colorbar(im0, ax=ax[i,j], orientation='horizontal')#, label=label)
ax[0,0].text(-45,-0.5, 'thermocline waters', fontsize=8)
ax[0,0].text(-45,-1.0, 'AAIW', fontsize=8)
ax[0,0].text(-45,-2.5, 'NADW', fontsize=8)
ax[0,0].text(-45,-4.5, 'AABW', fontsize=8)
dS0dt = r'$\frac{d}{dt}S_0=$'
ax[1,0].text(-52,-5.3, f'{dS0dt} {xr_linear_trend(S0t).values*365*100:4.3f} psu/100yr', fontsize=8)
plt.savefig(f'{path_results}/Mov/SALT_VVEL_trends')

In [ ]:
S0t= (section_rcp.SALT*ds.DXT*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(section_rcp.SALT>0)*ds.dz).sum(dim=['nlon','z_t'])

In [ ]:
S0t_trend = (section_rcp_SALT_trend*ds.DXT*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(section_rcp.SALT>0)*ds.dz).sum(dim=['nlon','z_t'])

In [ ]:
(365*100*S0t_trend).plot()

In [ ]:
plt.plot(S0t.time/365,S0t)
plt.plot(S0t.time/365,xr_lintrend(S0t), ls='--')
plt.title(r'$S_0(t)$')
plt.ylabel('SALT [psu]')
plt.ylabel('time [year]')
plt.text(2030, 34.85, f'{xr_linear_trend(S0t).values*365*100:4.3f} psu/100yr', c='C1')
plt.savefig(f'{path_results}/')

## Salinity bias
_Mecking et al. (2017)_ point out that the CMIP5 $M_{ov}$ values are influenced by salinity biases, so it is of interest to quantify these in the CESM simulations.

In [ ]:
dsh = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
dsl = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)

In [ ]:
for ds in [dsh, dsl]:
    S0y = (ds.SALT*ds.DXT*ds.dz).where(ds.REGION_MASK==6).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.REGION_MASK==6)*ds.dz).where(ds.SALT>0).sum(dim=['nlon','z_t'])
    plt.figure()
    S0y.plot()

In [ ]:
# how does this look like in OBS?
SALT_mean_low  = xr.open_dataarray(f'{path_prace}/EN4/EN4_mean_salinity_low.nc')
((SALT_mean_low*ds.DXT*ds.dz).where(ds.REGION_MASK==6).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.REGION_MASK==6)*ds.dz).where(ds.SALT>0).sum(dim=['nlon','z_t'])).plot()
print(((SALT_mean_low*ds.DXT*ds.dz).where(ds.REGION_MASK==6).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.REGION_MASK==6)*ds.dz).where(ds.SALT>0).sum(dim=['nlon','z_t']))[:250].mean(dim='nlat'))

In [ ]:
dsh.DXU.where(Atl_MASK_ocn).std(dim='nlon').plot()

In [ ]:
ds_ocn = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)
ds_low = xr.open_dataset(file_ex_ocn_lpd, decode_times=False)
RMASK_ocn = xr.open_dataarray(file_RMASK_ocn)
RMASK_low = xr.open_dataarray(file_RMASK_ocn_low)
Atl_MASK_ocn = xr.DataArray(np.in1d(RMASK_ocn, [6,8,9]).reshape(RMASK_ocn.shape),
                            dims=RMASK_ocn.dims, coords=RMASK_ocn.coords)
Atl_MASK_low = xr.DataArray(np.in1d(RMASK_low, [6,8,9]).reshape(RMASK_low.shape),
                            dims=RMASK_low.dims, coords=RMASK_low.coords)

In [ ]:
SALT_mean_low  = xr.open_dataarray(f'{path_prace}/EN4/EN4_mean_salinity_low.nc')
SALT_mean_high = xr.open_dataarray(f'{path_prace}/EN4/EN4_mean_salinity_high.nc')

In [ ]:
def calc_Fov_Faz_Sov_Saz_Se(ds, SALT=None, MASK=None):
    """ calculate F_{ov/az}, S_{ov,az} 
    
    ds   .. dataset with geometry fields dz and DXU
    SALT .. if None: taken from ds; else external SALT field for bias corrected calculation
    MASK .. which 
    
    """
    S0 = 35.
    if SALT is None:  SALT = ds.SALT
    if MASK is None:  MASK = ds.REGION_MASK.where(ds.REGION_MASK==6)
    SALT = SALT.where(MASK)
    VVEL = ds.VVEL.where(MASK)
    
    DXU_intx_z  = ds.DXU.where(MASK).where(SALT>0).sum(dim='nlon', skipna=True)  # zonal length of section (z)
    SALT_mean_z = (ds.DXU*SALT).sum(dim='nlon', skipna=True)/DXU_intx_z          # zonal mean salinity     (z) [g/kg]
    VVEL_intx_z = (ds.DXU*VVEL).sum(dim='nlon', skipna=True)                     # zonal integral velocity (z) [cm^2/s]
    VVEL_mean_z = VVEL_intx_z/DXU_intx_z                                         # zonal mean velocity     (z) [cm/s]
    SALT_prime_x_z = (SALT - SALT_mean_z)                                        # azonal salt component (x,z) [g/kg]
    VVEL_prime_x_z = (VVEL - VVEL_mean_z)                                        # azonal velocity comp. (x,z) [cm/s]
    
    Fov = ( -1/S0*(( VVEL_intx_z * (SALT_mean_z - S0) )*ds.z_t).sum(dim='z_t'))/1e12  # 1 Sv = 1e12 cm^3/s
    Sov = ((VVEL_intx_z*SALT_mean_z)*ds.z_t).sum(dim='z_t')*rho_sw/1e9            # 1 kg/s = rho_w * 1e-9 g/kg cm^3/s
    
    vSp = (ds.DXU*ds.DZT*1e2*VVEL_prime_x_z*SALT_prime_x_z).sum(dim=['nlon','z_t'])   # [cm^3/s * g/kg]
    Saz = vSp*rho_sw/1e9
    Faz = -vSp/1e12/S0
    
    Se = (ds.DXU*ds.DZT*1e2*(ds.VNS.where(MASK)*ds.DYT - VVEL*SALT)).sum(dim=['nlon','z_t'])*rho_sw/1e9
    Fov.name, Faz.name, Sov.name, Saz.name, Se.name = 'Fov', 'Faz', 'Sov', 'Saz', 'Se'
    ds = xr.merge([Fov, Faz, Sov, Saz, Se])
    return ds

In [ ]:
DZT

In [ ]:
for i, run in enumerate(['ctrl', 'lpd']):
#     if i==0:  continue
    DXU = ([dsh, dsl][i]).DXU
    DYT = ([dsh, dsl][i]).DYT
    dz = ([dsh, dsl][i]).dz
    DZT = xr_DZ(domain=['ocn', 'ocn_low'][i], grid='T')
    DZT.name = 'DZT'
    MASK = [Atl_MASK_ocn, Atl_MASK_low][i]
    for y in tqdm_notebook(range(5)):
        y_ = [200, 500][i]+y    
        SALT_VNS_UES = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_SALT_VNS_UES_0{y_}.nc', decode_times=False)
        UVEL_VVEL = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_UVEL_VVEL_0{y_}.nc', decode_times=False)
        ds = xr.merge([SALT_VNS_UES, UVEL_VVEL, DYT, DXU, DZT, dz])
        dso_ = calc_Fov_Faz_Sov_Saz_Se(ds, MASK=MASK)
        dsc_ = calc_Fov_Faz_Sov_Saz_Se(ds, MASK=MASK, SALT=[SALT_mean_high, SALT_mean_low][i])
        if y==0:  dso, dsc = dso_, dsc_
        else:     dso, dsc = xr.concat([dso, dso_], dim='time'), xr.concat([dsc, dsc_], dim='time')
    dso.to_netcdf(f'{path_prace}/{run}/FW_SALT_fluxes_{run}.nc')    
    dsc.to_netcdf(f'{path_prace}/{run}/FW_SALT_fluxes_bias_corrected_{run}.nc')

In [ ]:
f, ax = plt.subplots(2,2, figsize=(12,8), sharex='col', sharey='row')
ov, az, eddy = r'$_{ov}$', r'$_{az}$', r'$_{eddy}$'
for i, run in enumerate(['ctrl', 'lpd']):
    ax[0,i].set_title(['HIGH','LOW'][i])
    ax[i,0].set_ylabel(['northward FW transport [Sv]','northward Salt transport [kg/s]'][i])
    for j in range(2):
        ax[i,j].axhline(0, c='k')
        ax[i,j].set_xlim((-34.5,70))
        ax[i,j].grid()
    lats = [approx_lats('ocn'), dsl.TLAT.where(Atl_MASK_low).mean(dim='nlon', skipna=True)][i]
    dso = xr.open_dataset(f'{path_prace}/{run}/FW_SALT_fluxes_{run}.nc', decode_times=False)
    dsc = xr.open_dataset(f'{path_prace}/{run}/FW_SALT_fluxes_bias_corrected_{run}.nc', decode_times=False)
    
    # original
    ax[0,i].plot(lats, dso.Fov.mean('time')                                         , c='C0', label=f'F{ov}')
    ax[0,i].plot(lats, dso.Faz.mean('time')                                         , c='C1', label=f'F{az}')
    ax[0,i].plot(lats, dso.Fov.mean('time')+dso.Faz.mean('time')                    , c='C3', label=f'F{ov}+F{az}')
    ax[1,i].plot(lats, dso.Sov.mean('time')                                         , c='C0', label=f'S{ov}')
    ax[1,i].plot(lats, dso.Saz.mean('time')                                         , c='C1', label=f'S{az}')
    ax[1,i].plot(lats, dso.Se .mean('time')                                         , c='C2', label=f'S{eddy}')
    ax[1,i].plot(lats, dso.Sov.mean('time')+dso.Saz.mean('time')+dso.Se.mean('time'), c='C3', label=f'S{ov}+S{az}+S{eddy}')
    
    # bias corrected
    # VNS is not bias corrected, so this term does not make sense
    ax[0,i].plot(lats, dsc.Fov.mean('time')                     , c='C0', ls='--', lw=.5, label=f'F{ov} corr.')
    ax[0,i].plot(lats, dsc.Faz.mean('time')                     , c='C1', ls='--', lw=.5, label=f'F{az} corr.')
    ax[0,i].plot(lats, dsc.Fov.mean('time')+dsc.Faz.mean('time'), c='C3', ls='--', lw=.5, label=f'F{ov}+F{az} corr.')
    ax[1,i].plot(lats, dsc.Sov.mean('time')                     , c='C0', ls='--', lw=.5, label=f'S{ov} corr.')
    ax[1,i].plot(lats, dsc.Saz.mean('time')                     , c='C1', ls='--', lw=.5, label=f'S{az} corr.')
    ax[1,i].plot(lats, dsc.Sov.mean('time')+dsc.Saz.mean('time'), c='C3', ls='--', lw=.5, label=f'S{ov}+S{az} corr.')
#     ax[1,i].set_ylim((-1.5e8,1.5e8))
    ax[1,i].set_xlabel(r'latitude [$^\circ$N]')
    for j in range(2):  ax[j,i].legend(ncol=2)
plt.savefig(f'{path_results}/Mov/Mov_lat_ctrl_lpd')

### Salt bias corrected meridional FW/SALT fluxes.

### How well can we approximate the FW eddy term with averaging monthly fields to a yearly field?

In [ ]:
for run in ['ctrl', 'lpd']:
    SALT_VNS_UES = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_SALT_VNS_UES_0200.nc', decode_times=False)
    UVEL_VVEL = xr.open_dataset(f'{path_prace}/{run}/ocn_yrly_UVEL_VVEL_0200.nc', decode_times=False)
    ds = xr.merge([SALT_VNS_UES, UVEL_VVEL])
    

### What is the error inducesd due to the grid distortion?